In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.ml.feature import RegexTokenizer, NGram
from collections import defaultdict

spark = SparkSession.builder.appName("TrigramPartial").getOrCreate()
tokenizer = RegexTokenizer(inputCol="value", outputCol="words", pattern="\\W+", toLowercase=True)
ngram = NGram(n=3, inputCol="words", outputCol="trigrams")

def get_trigram_counts_for_chunk(chunk_text):
    rdd = spark.sparkContext.parallelize([chunk_text])
    df = rdd.map(lambda x: (x,)).toDF(["value"])
    tokenized = tokenizer.transform(df)
    ngramData = ngram.transform(tokenized)
    trigram_df = ngramData.select(explode(col("trigrams")).alias("trigram"))
    trigram_count_df = trigram_df.groupBy("trigram").count()
    partial_counts = trigram_count_df.collect()
    d = {}
    for row in partial_counts:
        d[row["trigram"]] = row["count"]
    return d

CHUNK_SIZE = 10000
accum_counts = defaultdict(int)
with open("shared/hw2/internet_archive_scifi_v3.txt", "r", encoding="utf-8", errors="ignore") as f:
    while True:
        data = f.read(CHUNK_SIZE)
        if not data:
            break
        chunk_dict = get_trigram_counts_for_chunk(data)
        for trig, cnt in chunk_dict.items():
            accum_counts[trig] += cnt

r = sorted(accum_counts.items(), key=lambda x: x[1], reverse=True)[:10]
for t, c in r:
    print(t, c)

spark.stop()


Py4JJavaError: An error occurred while calling o280.defaultParallelism.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServer